### Final Propensity matching results

In [1]:
%reset -f

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext Cython

In [3]:
%reload_ext Cython

In [4]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [5]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
from data_functions import make_date_to_MYR, make_MYR_to_date
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )
import data_process
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import models, layers
import warnings
warnings.filterwarnings('ignore')
from path import col_seq_can_all, col_seq_can 
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [6]:
# #setup cython
# #for cython build if needed -> cd to /modules folder and then run
# %cd ..
# !python3 setup.py build_ext --inplace
# %cd ./causal_data_creation/canary_application/

In [7]:
%config Completer.use_jedi = False

### Load pre matching data

In [8]:
pp_str = 'Pre_Matching_DATA/'
Path("./"+pp_str).mkdir(exist_ok=True)

In [9]:
# %%time
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/CANARY_PAPER/DATA/Pre_Matching_DATA ./Pre_Matching_DATA --recursive;
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/CANARY_PAPER/DATA/tempHybridSelectionMoreData/random.p ./Pre_Matching_DATA/random.p;

In [10]:
treated = pd.read_pickle(pp_str+'treated2.p')
controls = pd.read_pickle(pp_str+'controls.p')
pmpm =pd.read_pickle(pp_str+'pmpm.p')
random_controls = pd.read_pickle(pp_str+'random.p')

In [11]:
pmpm.MBR_GNDR.replace({"F":0,"M":1}, inplace=True)

In [12]:
treated.shape, treated.MYR.min(), treated.MYR.max()

((26040, 91), 201710, 202107)

In [13]:
controls.shape, controls.MYR.min(), controls.MYR.max()

((9582196, 91), 201701, 202107)

In [14]:
pmpm.shape, pmpm.MYR.min(), pmpm.MYR.max()

((9650961, 91), 201701, 202107)

# Check continuity and take average of 1 year history

### Treated

In [15]:
# treated_history = pmpm[pmpm.PERS_ID.isin(treated.PERS_ID)]
# treated.drop(['MEDICAID_LOB'], axis=1, inplace=True)
# treated['MYR_12'] = treated.MYR.apply(data_process.get_one_year_before, month_offset=12)
# treated_history.MBR_GNDR.replace({"F":0, "M":1}, inplace=True)

In [16]:
# %%time
# treated_aggregate = pd.DataFrame(columns=col_seq_can_all)
# for i, (index, row) in tqdm(enumerate(treated.iterrows())):
#     pid = row.PERS_ID
#     myr = row.MYR
#     old_myr = row.MYR_12
#     history = treated_history.query("PERS_ID==@pid & MYR>=@old_myr & MYR<=@myr")
#     if history.shape[0]>=10:
#         agg_data = history[col_seq_can_all].mean()
#         agg_data.PERS_ID = str(pid)
#         agg_data.MYR     = int(myr)
#         treated_aggregate = pd.concat([treated_aggregate, pd.DataFrame(agg_data).T], axis=0)

In [17]:
# treated_aggregate.PERS_ID = treated_aggregate.PERS_ID.astype(str)
# treated_aggregate.MYR = treated_aggregate.MYR.astype(int)

In [18]:
#treated_aggregate.to_pickle("treated_aggregate_filtered_final.p")

In [19]:
# treated_history = pmpm[pmpm.PERS_ID.isin(treated.PERS_ID)]
# test = treated_history.set_index(['PERS_ID','MYR']).rolling(12, min_periods=10).mean().dropna(how='any')
# test[test.index.isin(treated.set_index(['PERS_ID','MYR']).index)].reset_index()

In [20]:
treated_aggregate = pd.read_pickle("treated_aggregate_filtered_final.p") #goes into matching as tr_agg

In [21]:
treated_aggregate.groupby('PERS_ID').apply(lambda x: pd.Series(x.iloc[0])).reset_index(drop = True)

PERS_ID     MYR  AGE_AT_MIDMONTH  LOB  MBR_GNDR    ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR    CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0     2602482  201908        80.769231  1.0       0.0   278.769231       0.0       0.0      0.0           0.0                0.0     0.0        0.0             1.000000      0.000000        0.0       0.000000        0.0         0.0  0.000000               0.0     0.000000                0.0                0.0                   0.0      0.0                 0.0       0.000000                      0.0                0.0     0.000000         1.000000   3.307692                    0.0          0.0              0.0     0.846154           1.000000               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0           0.538462         0.923077             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0    0.000000   
1     2603072  202001        81.076923  1.0       0.0   267.230769       0.0       0.0      0.0           0.0                0.0     0.0        0.0             0.538462      0.000000        0.0       0.000000        0.0         0.0  0.000000               0.0     0.000000                0.0                0.0                   0.0      0.0                 0.0       0.000000                      0.0                0.0     0.000000         0.000000   2.538462                    0.0          0.0              0.0     0.000000           0.000000               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0           1.000000         1.000000             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0    0.000000   
2     2606370  202010        81.076923  1.0       0.0   850.923077       0.0       0.0      0.0           0.0                0.0     0.0        0.0             1.000000      0.076923        0.0       1.000000        0.0         0.0  0.000000               0.0     0.153846                0.0                0.0                   0.0      0.0                 0.0       0.923077                      0.0                0.0     0.000000         0.000000   5.153846                    0.0          0.0              0.0     0.000000           0.000000               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0           0.000000         0.000000             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0    1.000000   
3     2608221  202

In [22]:
treated_original = treated[treated.set_index(['PERS_ID', 'MYR']).index.isin(treated_aggregate.set_index(['PERS_ID', 'MYR']).index)] #goes into matching as treated

In [23]:
model = models.load_model(pp_str+"TF_Latent_Model_m6_Canary_stacked_logfixed", compile=False)

In [24]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [25]:
onset = "CC_DIABETES"
cols_to_keep = ["PERS_ID", "MYR", 'AGE_AT_MIDMONTH', 'MBR_GNDR', onset]

In [26]:
# treated_latent = pd.DataFrame(model.predict(treated_aggregate.loc[:,'AGE_AT_MIDMONTH':]), columns=latent_cols)
# treated_latent.insert(0, 'PERS_ID', treated_aggregate.PERS_ID)
# treated_latent.insert(1, 'MYR', treated_aggregate.MYR)

### Pick Columns for matching

In [27]:
columns_to_match =  ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY', 
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6','LOB']

all_columns1 = list(set(col for col in pmpm.columns)-set(['PERS_ID', 'MYR','MEDICAID_LOB',
                                                         'IP_BIN', 'ER_BIN', 'ACUTE', 'IP_C', 'ER_C', 'ACUTE_C','IP_BIN2', 'ER_BIN2']))

all_columns2 = list(set(all_columns1)-set(['IP_BIN6', 'ER_BIN6', 'LOB']))

In [ ]:
%%time

#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
#hier_cols = ['IP_BIN2', 'ER_BIN2', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']
match_on = latent_cols
hier_cols = ['IP_BIN6', 'ER_BIN6']
hier_range = [1,1]
exact_match_dict = {'IP_BIN6':[0,1,2], 'ER_BIN6':[0,1,2], 'LOB':None}
nnr_func_args = {'tr_agg':treated_aggregate.copy(), 'pmpm':pmpm.copy(), 'latent_cols':latent_cols, 'model':model, 'match_on':match_on, 'hier_cols':hier_cols,
                        'hier_range':hier_range, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': True, 'exact_match_dict': exact_match_dict}
matched0, matched_data0 = matched_population(treated_original.copy(), 
                                           controls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)

Mathcing Function: nearest_neighbor_func
hERE
After filtering control ids are:328522 before they were:354792


0it [00:00, ?it/s]

IP_BIN6 [0, 1, 2]
ER_BIN6 [0, 1, 2]
LOB None


6it [00:26,  4.45s/it]

####  In addition to the previously created matched groups create a random group

In [ ]:
Path('./tempHierALL').mkdir(exist_ok = True)
p = './tempHierALL/'
matched_data0.to_pickle(p+'match0_l.p')
tr2.to_pickle(p+'treated0_l.p')

In [ ]:
matched_data0_o = controls[controls.set_index(["PERS_ID", "MYR"]).index.isin(matched_data0.set_index(["PERS_ID", "MYR"]).index)].sort_values(by=["PERS_ID", "MYR"])

In [ ]:
matched_data0_o.to_pickle(p+"match0_o.p")

In [ ]:
rndm.to_pickle(p+"random_l.p")

In [ ]:
pmpm2 = pmpm
try:
    pmpm2.MBR_GNDR.replace(to_replace = {'F':0, 'M':1}, inplace = True)
except:
    pass

In [ ]:
from utils import statistics_first_month, p_val, stat_matrices 

### List containing all numerical columns

In [ ]:
all_columns = list(set(col for col in pmpm.columns)-set(['PERS_ID', 'MYR','MEDICAID_LOB']))

### Stats First Month(Treated and Matched Controls)

In [ ]:
%%time
tr_desc0, treated_first_month = statistics_first_month(treated2, all_columns, return_array = True)
con_desc0, controls_first_month = statistics_first_month(matched_data0_o, all_columns, return_array = True)

### Stats First Month(General Pop and Random)

In [ ]:
%%time
pmpm_desc = statistics_first_month(pmpm2, all_columns)
random_desc = statistics_first_month(random_controls, all_columns)

In [ ]:
%%time
conds_first, full_first = stat_matrices(tr_desc0, con_desc0, pmpm_desc, random_desc, columns = columns_to_match)
conds_firstAllCols, full_firstAllCols = stat_matrices(tr_desc0, con_desc0, pmpm_desc, random_desc, columns = all_columns)

## Statistics on matching date

In [ ]:
conds_first.round(3)

In [ ]:
full_first

### Statistics Over all dates (treated, controls)

In [ ]:
%%time
tr_desc0A = treated2[all_columns].describe().loc[['mean', 'std', 'count']].T
tr_desc0A['count'] = tr_desc0A['count'].astype(int)

con_desc0A = matched_data0_o[all_columns].describe().loc[['mean', 'std', 'count']].T
con_desc0A['count'] = con_desc0A['count'].astype(int)

### Statistics Over all dates (general population, Random)

In [ ]:
%%time
pmpm_descA = pmpm2[all_columns].describe().loc[['mean', 'std', 'count']].T
pmpm_descA['count'] = pmpm_descA['count'].astype(int)

random_descA = random_controls[all_columns].describe().loc[['mean', 'std', 'count']].T
random_descA['count'] = random_descA['count'].astype(int)

In [ ]:
%%time
conds, full = stat_matrices(tr_desc0A, con_desc0A, pmpm_descA, random_descA, columns = columns_to_match)
conds_AllCols, full_AllCols = stat_matrices(tr_desc0A, con_desc0A, pmpm_descA, random_descA, columns = all_columns)

In [ ]:
conds.round(3)

#### Distribution Plots For Treated and Matched Controls
1)  Sick(S) Treated versus  Sick Control groups  
2) Healthy(H) Treated versus Healthy Control Groups

### Matrices and Figures for the paper

In [ ]:
columns_to_match

In [ ]:
mapping = dict(zip(columns_to_match, ['Age', 'Total Cost', 'Gender', 'Tobacco Use', 'Pressure', 'Obesity', 
                                     'Hypertension', 'Hypothyroid', 'Disease Count','Acute Care 2 Months', 'Acute Care 6 Months',
                                      'Inpatient Visits 6 Months', 'Emergency Vistits 6 Months', 'Line of Bussiness']))
mapping2 = dict(zip(all_columns, all_columns))
#mapping = dict(zip(columns_to_match, columns_to_match))
new_names = list(mapping.values())

In [ ]:
base_path =  './results/hier_matching_ALL/'
p = Path(base_path)
p.mkdir(parents = True, exist_ok = True)

In [ ]:
import matplotlib as mpl
sns.set_context(context = 'paper')
mpl.style.use('seaborn-paper')

In [ ]:
from diabetes_onset_plotter import plot_hist, save_arrays, save_arrays2


#### remove %%capture to see output

In [ ]:
%%capture

dist_plot_args = {'hist':True}
axes = plot_hist(treated_first_month.copy(), 
                 controls_first_month.copy(), mapping2, ['ALLOW_AMT'] , 
                 data3 = None, 
                 save_path = base_path+'figures/distributions_all/',
                dist_plot_args = dist_plot_args)

In [ ]:
%%capture
dist_plot_args = {'hist':True}
axes = plot_hist(treated_first_month.copy(), 
                 controls_first_month.copy(), mapping, ['ALLOW_AMT'] , 
                 data3 = None, 
                 save_path = base_path+'figures/distributions_selection/',
                dist_plot_args = dist_plot_args)

### Save enhanced stats arrays(these will be used at the end)
Arrays with: Statistics in first day, statistics  in the span of monthsm statistiocs from selected columns and all columns

In [ ]:
arrays = [conds_first, full_first, conds_firstAllCols, full_firstAllCols,
         conds, full, conds_AllCols, full_AllCols]
names = ['conds_first', 'full_first', 'conds_firstAllCols', 'full_firstAllCols',
         'conds', 'full', 'conds_AllCols', 'full_AllCols']
save_arrays2(arrays, names, base_path+'tables/stats/')

# Latent Space Stat

In [ ]:
latent_cols

In [ ]:
%%time
tr_desc0_l, treated_first_month_l = statistics_first_month(tr2, latent_cols, return_array = True)
con_desc0_l, controls_first_month_l = statistics_first_month(matched_data0, latent_cols, return_array = True)

In [ ]:
pmpm_lat = data_process.transform_features_with_NN(model, pmpm[col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

In [ ]:
%%time
pmpm_desc_l = statistics_first_month(pmpm_lat, latent_cols)
random_desc_l = statistics_first_month(rndm, latent_cols)

In [ ]:
%%time
conds_first_latent, full_first_latent = stat_matrices(tr_desc0_l, con_desc0_l, pmpm_desc_l, random_desc_l, columns = latent_cols)

In [ ]:
conds_first_latent

In [ ]:
full_first_latent

In [ ]:
conds_first_latent.to_csv('./results/hier_matching_ALL/tables/stats/conds_first_latent.csv', index=True, header=True)

In [ ]:
full_first_latent.to_csv('./results/hier_matching_ALL/tables/stats/full_first_latent.csv', index=True, header=True)

# Standardized Means Graph

In [ ]:
tr_desc0, treated_first_month = statistics_first_month(treated2, columns_to_match, return_array = True)

In [ ]:
con_desc0, controls_first_month = statistics_first_month(matched_data0_o, columns_to_match, return_array = True)

In [ ]:
un_controls_ = controls[columns_to_match].describe().loc[['mean', 'std', 'count']].T

In [ ]:
un_controls_all = controls[all_columns].describe().loc[['mean', 'std', 'count']].T

In [ ]:
un_controls_all.to_csv("unmatched_all_cols_stats.csv", index=True, header=True)

In [ ]:
len(controls)

In [ ]:
pooled = (tr_desc0['std']* 1604 + un_controls_['std']*len(controls))/(1604+len(controls))

In [ ]:
stdMeans = pd.DataFrame()
stdMeans['Treated'] = tr_desc0['mean']/pooled
stdMeans['Controls'] = un_controls_['mean']/pooled
stdMeans['MControls'] = con_desc0['mean']/pooled

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,7))
ax[0].plot([0,3], [0, 3])
ax[1].plot([0,3], [0, 3])
ax[0].set_xlabel('Treated Standarized Means')
ax[0].set_ylabel('Unmatched Controls Standarized Means')
sns.regplot(stdMeans['Treated'].values, stdMeans['Controls'].values,scatter = False, ax = ax[0],line_kws={"color": "red","linestyle" : '--'
})

ax[0].scatter(stdMeans['Treated'], stdMeans['Controls'])
ax[1].scatter(stdMeans['Treated'], stdMeans['MControls'])
ax[1].set_xlabel('Treated Standarized Means')
ax[1].set_ylabel('Matched Controls Standarized Means')
sns.regplot(stdMeans['Treated'].values, stdMeans['MControls'].values, scatter = False,ax = ax[1], line_kws={"color": "red","linestyle" : '--'
})
for i, name in enumerate(list(stdMeans.index)):
    ax[0].annotate(name, (stdMeans['Treated'].iloc[i],stdMeans['Controls'].iloc[i]), fontsize = 'small',
                   textcoords = 'offset points',xytext = (-10,10))
    ax[1].annotate(name, (stdMeans['Treated'].iloc[i],stdMeans['MControls'].iloc[i]),fontsize = 'small',
                   textcoords = 'offset points', xytext = (-10,10))

fig.show()

# Ratio Stats

In [ ]:
disease_cols = [cols for cols in treated2.columns if "CC" in cols and cols!='CC_DX_CNT']
other_binary_cols = ['MBR_GNDR', 'LOB', 'PREGNANCY', 'IP_BIN', 'ER_BIN', 'ACUTE']
binary_cols = other_binary_cols + disease_cols 
restricted_cols = ['PERS_ID', 'MYR', 'MEDICAID_LOB']
continuous_cols = [cols for cols in treated2.columns if cols not in binary_cols + restricted_cols]

In [ ]:
def calculate_continuous_d(t, c):
    Xt = np.mean(t)
    Xc = np.mean(c)
    St = np.std(t)
    Sc = np.std(c)
    d = np.abs((Xt-Xc)/(np.sqrt((St**2 + Sc**2)/2)))
    return d

In [ ]:
def calculate_binary_d(t, c):
    Pt = np.mean(t)
    Pc = np.mean(c)
    d = np.abs((Pt-Pc)/(np.sqrt((Pt*(1-Pt)+Pc*(1-Pc))/2)))
    return d

In [ ]:
d_metric_before = pd.DataFrame()
for col in treated2.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated2[[col]].values, controls[[col]].values)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated2[[col]].values, controls[[col]].values)
        d_metric_before.loc[0,col] = d

In [ ]:
d_metric_before.T.to_csv("d_metric_before.csv", header=True, index=True)

In [ ]:
d_metric_before.T[d_metric_before.T[0]<0.1].size

In [ ]:
d1 = d_metric_before.loc[:,:'LOB']

In [ ]:
d1.T[d1.T[0]<0.1].size

In [ ]:
d_metric_after = pd.DataFrame()
for col in treated2.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated2[[col]].values, matched_data0_o[[col]].values)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated2[[col]].values, matched_data0_o[[col]].values)
        d_metric_after.loc[0,col] = d

In [ ]:
d_metric_after.T.to_csv("d_metric_after.csv", header=True, index=True)

In [ ]:
d_metric_after.T[d_metric_after.T[0]<0.1].size

In [ ]:
d2 = d_metric_after.loc[:,:'LOB']

In [ ]:
d2.T[d2.T[0]<0.1].size

In [ ]:
ratio = d_metric_after/d_metric_before
ratio

In [ ]:
ratio.T[ratio.T[0]>=1.0].size

In [ ]:
ratio.T.to_csv("ratio.csv", header=True, index=True)